<a href="https://colab.research.google.com/github/Yuns-u/Boston_Housing_predict_practice/blob/main/Practice_Boston_Housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practice Deep Learning on regression problem

회귀문제를 딥러닝모델을 통해 해결해보고자 한다.
해결하고자 하는 문제는 아래와 같다.

1. Boston의 주택가격을 예측하는 딥러닝 모델을 만들어보기
2. 딥러닝이 아닌 머신러닝모델을 사용해서 같은 문제를 풀어보고 그 결과를 비교해보기

# 데이터 불러오기 및 전처리

In [ ]:
# 필요한 모듈 불러오기
import tensorflow as tf

In [ ]:
# 데이터 불러오기
boston_housing = tf.keras.datasets.boston_housing
(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

In [ ]:
# 각 데이터셋의 shape 확인하기 : feature 수에 비해 데이터의 규모가 작은 것으로 보인다.
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
X_train 
#numpy.array형식이라 pandas.DataFrame처럼 상위 몇 개를 보는 방법을 사용할 수 없었다.
#하지만 pandas의 메모리 소모량을 생각했을 때 불러오지 않고 처리를 해보고 싶기도하고 pandas를 사용하지 않고 head처럼 처리할 수 있는 방법이 있을 것 같다.

In [ ]:
y_train

In [ ]:
# 데이터 정규화하기
# 데이터 정규화에 필요한 모듈 불러오기
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train) #훈련해야하는 수치들을 정규화한다.
X_test = scaler.transform(X_test) #훈련데이터가 정규화되었으므로 예측의 기반이 될 테스트데이터셋도 정규화해준다.

# 신경망을 통해 regression 문제 해결하기

In [ ]:
#모델 만들기
model = tf.keras.models.Sequential([
 tf.keras.layers.Dense(64, activation='relu', input_dim=13),
 tf.keras.layers.Dropout(0.2),
 tf.keras.layers.Dense(16, activation='relu'),
 tf.keras.layers.Dense(1, activation='relu')
])

In [ ]:
#모델에 컴파일러 적용하여 옵티마이저, 손실함수 등 설정해주기
model.compile(optimizer='adam', 
              loss='mse',
              metrics=['accuracy','mse','mae'])

In [ ]:
#학습 실행해주기
history = model.fit(X_train, y_train, epochs=30, batch_size=30)

In [ ]:
#모델평가하기
#R2 값으로 모델의 설명력 확인해보기
from sklearn.metrics import r2_score

y_pred = model.predict(X_test)

r2_score(y_test, y_pred)

# 신경망이 아닌 머신러닝모델을 사용해서 regression 문제 해결하기

In [ ]:
#선형회귀모델로 머신러닝모델 만들기

#필요한 모듈 불러오기
from sklearn.linear_model import LinearRegression #모델 만들기
from sklearn.metrics import mean_squared_error #위의 신경망과 비교하기 위한 모듈
from sklearn.metrics import mean_absolute_error #위의 신경망과 비교하기 위한 모듈

#선형회귀모델 만들기
model = LinearRegression()
#모델 학습시키기: 여러 feature들로 구성되어있으므로 다중선형회귀모델일 것이다.
model.fit(X_train, y_train)
#예측하기
y_pred = model.predict(X_test)

#예측값과 실제값 비교하여 평가하기
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE : {mae}')
print(f'MSE : {mse}')
print(f'R2 : {r2}')

# 두 모델을 비교해보기

- 신경망모델의 r2값과 다중선형회귀모델의 r2값을 비교해보면 다중선형회귀모델의 값이 더 높기 때문에 이 회귀문제에 대한 데이터의 설명력은 다중선형회귀모델이 더 좋다고 할 수 있다.

- 신경망모델의 경우 30 epoche가 될 때 쯤에서야 다중선형회귀모델과 유사한 평가지표(mse, mae)값이 나온 것을 확인할 수 있다. `loss: 21.3019 - accuracy: 0.0000e+00 - mse: 21.3019 - mae: 3.3147`에서 알 수 있다.

- 신경망 모델에서 epoche가 크다고 해서 성능이 항상 좋게 나오는 것은 아니다. 어느정도 이상에서는 epoche 횟수를 늘리는 것이 유의미한 성능 향상을 불러오는 것이 아니기 때문이다.

- 자원 리소스 측면에서 신경망 모델을 쓰는 것이 더 비효율적일 수 있다.